
#  How compilers translate C programs


Enter your name and student ID.

 * Name:
 * Student ID:



* <font color="red">As this notebook is rather long, you may want to use the index shown by clicking the list icon on the left</font>



<a name="intro"> </a>
# 1. Introduction
* your final goal in the course is to build a compiler
* a compiler reads a program, builds a parse tree, and finally translates it to a _<font color="blue">machine code</font>_
* _<font color="blue">a machine code</font>_ is a series of instructions a processor can directly execute and is formatted in a way a machine can easily interpret (e.g., an instruction has a fixed number of bits, a fixed part of which encodes the kind of instructions)
* _<font color="blue">an assembly code</font>_ is practically the same as a machine code (a single assembly instruction almost directly corresponds to a single machine instruction) but is formatted as a text easier for humans to read
* it is common that a compiler actually generates an assembly code, which is then converted to a real machine code by another software (which you can consider as a stage in a compilation process), called _<font color="blue">an assembler</font>_
* before actually building a compiler, you need to know the assembly code of the target machine and at least roughly know how expressions and statements in the source language are mapped to assembly code
* even if you are not building a compiler, such a knowledge is useful in its own right to understand how programming languages work, to write an efficient program in them, and/or to diagnose your programs (especially for unsafe languages such as C or C++)
* to that end, the goal of this notebook is to look at and understand assembly code generated from C language and translate some C functions into assembly by hand



<a name="let_the_compiler_generate_assembly"> </a>
# 2. Let the compiler generate assembly code
* `gcc/g++` is a compiler for C/C++
* `-S` option instructs the compiler to emit assembly code and stop the compilation after that

In [ ]:
%%writefile call.c
long bbb();

long aaa() {
  return bbb() + 1;
}

In [ ]:
gcc -O3 -S call.c
cat call.s


* we choose 64 bit Intel x86 instruction set (called _<font color="blue">x86\_64</font>_ or _<font color="blue">amd64</font>_) as the target assembly code
 * this notebook teaches you the necessary part of x86_64 assembly. don't worry if you do not know about it
 * nobody can remember all x86_64 instructions. the point is to understand the minimum set of instructions necessary by a compiler and to be able to examine instructions corresponding to specific C expressions
 * [a quick reference](https://www.felixcloutier.com/x86/)
 * [a brief introduction by a lecture in Brown University](http://cs.brown.edu/courses/cs033/docs/guides/x64_cheatsheet.pdf)
 * [A complete reference by Intel](https://cdrdv2.intel.com/v1/dl/getContent/671143)

* below, you are going to learn what kind of expressions or statements are converted to what kind of assembly code, gradually changing functions from trivial ones to more substantial ones



<a name="assembly_basics"> </a>
# 3. CPUs and machine code (assembly code) basics
* before looking into assembly code, here is the basics of CPUs

<img src="img/cpu.svg" /> 

* a CPU or more precisely, a (virtual) core of a CPU, has a small number (a few to dozens) of storage, called _<font color="blue">registers</font>_ (a single CPU nowadays has a number of (virtual) cores in it. in this exercise we almost exclusively talk about a single virtual core)
* registers are almost never enough to hold data of the program, so most other data are put in _<font color="blue">the main memory</font>_
* you use an _<font color="blue">address</font>_ to specify the location in the memory you want to access.  an address is in fact just an integer.  you can think of it as being similar to an ordinary "address" in daily life.  it is not a complex string we use for our home addresses (e.g., 7-3-1 Hongo Bunkyo-ku, Tokyo), however.  it may be more similar to zip code, in this sense.
* a virtual core executes instructions one after another. instructions are also put in the memory.  after an instruction is executed, the instruction at the immediately next address is usually executed next
* an instruction typically does one of the following
 - (<font color="blue">load</font> instruction) : gets data from an address in the memory to a register
 - (<font color="blue">store</font> instruction) : gets data from a register to an address in the memory
 - (<font color="blue">arithmetic</font> instructions) :  computes a value from values on one to a few registers and put the result into a register 
 - (<font color="blue">jump</font> instruction) : changes the address from which the next instruction is taken
 - (<font color="blue">conditional</font> branch instruction) : changes the address from which the next instruction is taken depending on the result of recent operations (typically a comparison)
* some instructions combine the effect of two; for example, there is an instruction that takes one of its operands in a memory and computes a value based on it (and some other values)
* it may not be immediately clear to you how you can convert programs in high level languages to something like this, but the mapping is, at least conceptually, not as complex as you might think
* the mapping is roughly the following

| high-level languages       | assembly                      |
|----------------------------|-------------------------------|
| variable                   | register or memory            |
| operators in an expression | combination of arithmetic instructions (+ load/store as necessary) |
| if statements or loops     | instructions + conditional branches |

* after all, assembly is just another programming language that has many restrictions, some of which are
 * you cannot have arbitrary number of variables of names you choose; you instead only have a fixed number of variables of fixed names (i.e., registers) and one huge array (the entire address space)
 * you cannot arbitrarily nest expressions (e.g., a * x + b * y) and an instruction (say '+') can only take a constant or a variable as its operands; the above expression must be broken down into a series of instructions (s = a * x; t = b * y; u = s + t)
 * you do not have structured statements (if, while, for, etc.); you instead have to implement them with a combination of jumps and conditional branches
 * you do not have a function call or variables local to functions either; you instead have to implement them with a combination of jumps and a memory allocation scheme for local variables (stacks)
* with the above in mind, let's witness the the mapping between your language and assembly below



<a name="function_by_function"> </a>
# 4. A compilation is done on a function-by-function basis
* let's first compile four functions that do nothing


In [ ]:
%%writefile empty.c
void aaa() {  }

void bbb() {  }

void ccc() {  }

void ddd() {  }

In [ ]:
gcc -O3 -S empty.c
cat empty.s


observe

* you find strings aaa:, bbb:, ccc:, ddd: corresponding to function names
* they are called _<font color="blue">labels"</font>_, whose purpose is to name addresses in the instructions so you can conveniently reference them in other instructions (e.g., jump instructions)
* there come instructions for a function after its label

* assembly code for the four empty functions naturally look the same; let's extract what seems for `aaa:`
```
    .p2align 4
	.globl	aaa
	.type	aaa, @function
aaa:
.LFB0:
	.cfi_startproc
	endbr64
	ret
	.cfi_endproc
.LFE0:
	.size	aaa, .-aaa
```
* of which the actual instructions are the only 
```
	endbr64
	ret
```

* generally, lines starting from `.` do not represent actual instructions but directives to instruct the assembler in various ways such as "put the following instruction at addresses that are multiple of four" and you do not have to master them in depth (when you build a compiler, they are mostly boilerplate you can generate without deeply understanding them)

* what kind of assembly code `aaa` ended up translated, after all?  essentially, it is only this
```
aaa:  ## start of aaa
	endbr64
	ret
```

* `endbr64` is essentially a no-op, which is actually not necessary.  the reason why it is put despite it's doing nothing is explained the page ["What does the endbr64 instruction actually do?"](https://stackoverflow.com/questions/56905811/what-does-the-endbr64-instruction-actually-do)

* `ret` is a _<font color="blue">return instruction</font>_ and roughly speaking it ends the execution of a function
* a more detailed explanation beyond "it ends a function" will come shortly.  for now, you only have to understand that it is a kind of jump instruction and instructions that textually follow it will not be executed (without `ret` instruction here, the execution continues to instructions immediately below, which are bbb. a processor does not know which instructions constitute a function but just blindly execute instructions told to execute next)
* as this is a function that finishes without doing anything, it is natural that only `ret` instruction gets executed (except for the `endbr64` which is a no-op)
* the point here is not so much how an empty function is compiled, but that a translation from C to assembly is basically done on a function-by-function basis.  the basic issue is therefore _how to translate a single_ C function_ into a corresponding assembly code (sequence of instructions) 

* __points you learned:__ 
 1. for each C function, instructions corresponding to it are generated
 1. a label whose name is the same as the function name is put at the start of them




<a name="call_insn"> </a>
# 5. An instruction to call a function
* labels such as `aaa:`, `bbb:`, etc. are used for using (i.e., calling) these functions
* let's write a function aaa that calls bbb to see how it is done


In [ ]:
%%writefile call.c
long bbb();

long aaa() {
  return bbb() + 1;
}

In [ ]:
gcc -O3 -S call.c
cat call.s


* it is much more complex than the previous example, but for now, just notice the following
```
    call	bbb@PLT
```

* it is a kind of jump instruction called _<font color="blue">call instruction</font>_.  it is an instruction that "calls a function named `bbb`" = "transfers the execution to the instruction at the label `bbb`"
* you may wonder what is `@PLT` but you can ignore it for a moment (you may assume you always have it)
* when `bbb` executes `ret` instruction eventually, the control goes back to the caller, the instruction immediately after `call bbb@PLT` = `addq $8,%rsp`.  details of how it happens will be explained later.
* details aside, a call instruction jumps to the label of the specified function to effectively start execution of the function, which eventually executes a `ret` function, which gets the execution back to the instruction immediately after the call instruction

* __points you learned:__ 
 1. `call f@PLT` transfers the execution to the instruction at label `f`
 1. `ret` instruction transfers the execution to the caller (the instruction immediately after the call instruction)
 1. calling a function and continuing the the execution after its return is implemented in this way
 1. it therefore suffices to look at a function at a time when you examine the translation from C to assembly



* now we understand the execution goes back and forth between functions by `call` and `ret` instructions
* below, we are going to see how "values" are held and passed between functions
* there are various "values"; global variables, function return values, function parameters, local variables, etc.


<a name="args_and_return_value"> </a>
# 6. Parameters and return values
* for values to be passed via parameters and return values, they should be placed on some registers or somewhere in memory 
* first take a look at a function that takes a single integer (long) value as a parameter

<a name="mul_insn"> </a>
## 6-1. multiplication

In [ ]:
%%writefile mul_123.c
long mul_123(long x) {
  return 123 * x;
}

In [ ]:
gcc -O3 -S mul_123.c
cat mul_123.s


* looking at the generated code, the only instruction other than `endbr64` and `ret` is 

```
    imulq $123,%rdi,%rax
```
* this instruction does what `%rax = 123 * %rdi` would do in C (`%rax` and `%rdi` are both register names)
* you can deduce the following
 1. the first parameter `x` is on the register `%rdi`
 1. the return value is put on `%rax`
* note: `imulq` is an instruction name that comes from _<font color="blue">Integer MULtiply Quadword</font>_.  Quadword means that its operand is 8 bytes wide (C's `long` in 64 bit environment is 8 bytes)
* it may be a bit stretching to conclude that "the first parameter is always on `%rdi`" and "the return values is always `%rax`"; they must indeed be different if types (sizes) of them are different (e.g., larger than 8 bytes), but it is also very conceivable that there must be some uniform rules across functions
* a called function (say `g`) and the calling function (say `f`) are not necessarily compiled at the same time.  we don't know which other functions call `g` in future and `g` may be compiled again without recompiling `f`.  given these circumstances, it would be troubling if where `f` is supposed to pass values and where `g` is supposed to receive values are not agreed by some convention.  therefore, there is a rule with which one can know how a function passes its parameters and return value without looking inside the function.  such a rule is called _<font color="blue">calling convention</font>_ or _<font color="blue">application binary interface (ABI)</font>_.  here is the spec, [The x86-64 psABI version 1.0](https://github.com/hjl-tools/x86-psABI/wiki/x86-64-psABI-1.0.pdf). this notebook teaches you necessary elements, so you don't have to read it thoroughly

* __points you learned:__ 
 1. a first integer (long) parameter is passed via `%rdi` register
 1. an integer (long) return value is passed via `%rax` register



<a name="sub_insn"> </a>
## 6-2. subtraction
* let's confirm with another example that the first parameter is on `%rdi` and the return value on `%rax`


In [ ]:
%%writefile sub_123.c
long sub_123(long x) {
  return 123 - x;
}

In [ ]:
gcc -O3 -S sub_123.c
cat sub_123.s


* generated instructions

```
    movl $123,%eax
    subq %rdi,%rax
```

* `movl $123,%eax` sets 123 on `eax` register
* an astute student may wonder why it's `eax` not `rax`.  in fact, `eax` refers to the last 32 bits of `rax`. the above instruction sets the lower 32 bits of `rax` to 123 and the higher half to 0. it is therefore essentially `%rax = 123`
* `subq %rdi,%rax` does what `rax = rax - rdi` of C would do
* it is thus observed that here again x (the first parameter) is on `%rdi` and the return value on `%rax` 

<font color="blue">note:</font>
* many x86\_64 instructions have two operands (_op X,Y_) and perform an operation _Y = Y op X_
* the order is a bit confusing especially for subtraction (`subq` _X,Y_ computes _Y - X_, not _X - Y_)
* a rule to remember is the destination register is always on the right (_Y_) and `subq` _X,Y_ does _Y -= X_, which is _Y = Y - X_




* let's see that the caller also puts the first parameter on `%rdi` and receive the return value through `%rax`


In [ ]:
%%writefile f_mul_123.c
long f(long);

long f_mul_123() {
  return f(100) * 123;
}

In [ ]:
gcc -O3 -S f_mul_123.c
cat f_mul_123.s


* the first instruction 
```
    movl $100, %edi
```
sets  `%rdi` to 100 (the reason why it's `%edi` is the same as before. it actually sets `%rdi`'s upper 32 bits to 0 and the lower 32 bits to 100)

* the instruction after `call f@PLT`
```
    imulq $123, %rax, %rax
```
indicates that it expects that the return value is on `%rax`



<a name="div_insn"> </a>
## 6-3. division
* let's take a look at division and addition too


In [ ]:
%%writefile div_123.c
long div_123(long x) {
  return 123 / x;
}

In [ ]:
gcc -O3 -S div_123.c
cat div_123.s


```
    movl $123,%eax
    cqto
    idivq %rdi
```
* from the instruction name we can imagine `idivq %rdi` will be a division instruction, but it apparently takes only a single operand (`%rdi`)
* the fact of the matter is that the other operand (dividend) of `idivq` is the 128 bit number concatenating rdx (64 bit) and rax (64 bit)

* `idivq %rdi` performs the following (computes both quotient and remainder)
```
    (%rdx,%rax) = (%rdx:%rax % %rdi, %rdx:%rax / %rdi)
```
* that is, it divides `%rdx:%rax` by `%rdi` and puts the quotient on `%rax` and the remainder on `%rdx`, where `%rdx:%rax` is a 128 bit number whose higher 64 bits are `%rdx` and the lower 64 bits `%rax`



* details on idivq instruction can be found at [the Brown University course material](https://cs.brown.edu/courses/cs033/docs/guides/x64_cheatsheet.pdf)
* `cqto` stands for _<font color="blue">Convert Quadword To Octoword</font>_. it extends `%rax` to 128 bits and puts the upper/lower 64 bits on `%rdx` and `%rax`, respectively
* it can be seen again that `x` (the first parameter) is on `%rdi` and `%rax` holds the return value



<a name="add_insn"> </a>
## 6-4. addition

In [ ]:
%%writefile add_123.c
long add_123(long x) {
  return 123 + x;
}

In [ ]:
gcc -O3 -S add_123.c
cat add_123.s

* 
```
    leaq 123(%rdi),%rax
```
is essentially an addition instruction that performs

```
    %rax = 123 + %rdi
```
* the mysterious name `leaq` stands for _<font color="blue">Load Effective Address Quadword</font>_
* it is so called as it can be used to calculate the address 123 bytes ahead of the address in `%rdi`, which is exactly `(%rdi + 123)`.  note that it is not a memory access instruction despite it has _load_ in its name

* more naturally, we could do
```
    movl $123,%eax
    addq %rdi,%rax
```
but using only `addq`, it always takes two instructions, which may be why `leaq` is used (note: performance won't differ in any recent superscalar processors)



<a name="summary_so_far"> </a>
## 6-5. summary so far
* the first parameter of long (64 bit integer, to be more precise) type is passed via `%rdi`
* the return value of long type is passed via `%rax`

* arithmetic instructions on long types are as follows

| operation      | operand               | instruction (example)  |  semantics (in C-like expression)  |
|----------------|-----------------------|------------------------|------------------------------------|
| addition       | 64 bit integer (long) | addq \$123,%rax        |  %rax += 123                       |
| addition       | 64 bit integer (long) | leaq 123(%rdi),%rax    |  %rax = 123 + %rdi                 |
| subtraction    | 64 bit integer (long) | addq \$123,%rax        |  %rax -= 123                       |
| multiplication | 64 bit integer (long) | imulq \$123,%rdi,%rax  |  %rax = 123 * %rdi                 |
| division       | 64 bit integer (long) | idivq %rdi             | (%rdx,%rax) = (%rdx:%rax / %rdi, %rdx:%rax % %rdi) |

* when you want to know the instruction that does something, write a small function that only does it and see the output of the compiler



* In the problems on this page, you practice manually converting a simple example into an assembly code

<a name="problem_3x_plus_5"> </a>
# <font color="green"> Problem 1 :  3x+5</font>
* write a function `mul_3_add_5` that returns 3x+5 for a given input x (whose type is long) __in assembly__

* that is, the translation of the following C function
```
    long mul_3_add_5(long x) { return 3 * x + 5; }
```



* fill the following assembly function in assembly code (what follows ------- 解答をここに書く. write your answer here -------) with instructions and execute it

* it is saved with the name `mul_3_add_5.s` and compiled


In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile mul_3_add_5.s
	.file	"mul_3_add_5.c"
	.text
	.p2align 4,,15
	.globl	mul_3_add_5
	.type	mul_3_add_5, @function
mul_3_add_5:
.LFB0:
	.cfi_startproc
/* ------- 解答をここに書く. write your answer here ------- */

	.cfi_endproc
.LFE0:
	.size	mul_3_add_5, .-mul_3_add_5
	.ident	"GCC: (Ubuntu 7.4.0-1ubuntu1~18.04.1) 7.4.0"
	.section	.note.GNU-stack,"",@progbits



* then make an executable below


In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile check_mul_3_add_5.c
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
long mul_3_add_5(long);

int main(int argc, char ** argv) {
  long x = atol(argv[1]);
  long y = mul_3_add_5(x);
  assert(y == 3 * x + 5);
  printf("OK\n");
  return 0;
}

* execute the cell below; if you see three OK's and no errors, you are done

In [ ]:
BEGIN SOLUTION
END SOLUTION
gcc -o check_mul_3_add_5 -O3 check_mul_3_add_5.c mul_3_add_5.s

In [ ]:
BEGIN SOLUTION
END SOLUTION
./check_mul_3_add_5 10
./check_mul_3_add_5 20
./check_mul_3_add_5 30


#  If things do not go well
* If your program compiles but does not produce the correct answer, run it within a debugger (gdb)
* To that end, first compile the program with -g


In [ ]:
BEGIN SOLUTION
END SOLUTION
gcc -o check_mul_3_add_5 -O0 -g check_mul_3_add_5.c mul_3_add_5.s


* Go to terminal (login with SSH) and run the debugger
```
cd notebooks/pl07_compile_c
gdb check_mul_3_add_5
(gdb) break mul_3_add_5
(gdb) run
```

* This way, you can step into the mul_3_add_5 function
* You continue doing `step` to execute one instruction at a time
* At each instruction you can see the values in the registers by, e.g.,
```
(gdb) print $rax
```
or
```
(gdb) info registers
```

* Of course, you can use vscode or emacs for a better debugging experience


<a name="pointer"> </a>
# 7. Pointers
* if you have trouble understanding pointers in C language, read the following and the scales may fall from your eyes
* motto is "pointers are just integers and nothing more than that"

<a name="pointer_deref"></a>
## 7-1. pointer dereferencing

In [ ]:
%%writefile ptr_deref.c 
long long_ptr_deref(long * p) {
  return *p;
}

In [ ]:
gcc -O3 -S ptr_deref.c 
cat ptr_deref.s


* what is generated from `*p` seems 
```
    movq (%rdi),%rax
```
which is a _<font color="blue">load instruction</font>_ that uses the address in `%rdi` to read the eight bytes (the `q` of `movq` stands for quadword $=$ 8bytes) at that address and puts it on `%rax`. the following is therefore observed

* __points you learned:__
 1. a pointer parameter (`p`) is passed by `%rdi`, just like an integer parameter
 1. a pointer value of C is in fact an "address", which is merely an integer in the assembly code level
 1. dereferencing a pointer `p` (*p) refers to the value stored at the address in pointer `p`.  a load instruction is therefore used to extract the value



* various superficially different expressions in C all end up reading certain addresses
* referencing an array element, for example


In [ ]:
%%writefile array_index_long.c
long array_index_long(long * p) {
  return p[0] + p[10];
}

In [ ]:
gcc -O3 -S array_index_long.c
cat array_index_long.s


* what is generated for
```
    p[0] + p[10];
```
appears 
```
    movq 80(%rdi),%rax
    addq (%rdi),%rax
```
* the first instruction
```
    movq 80(%rdi), %rax
```
reads eight bytes from the address (the value in `%rdi` + 80) and puts it in `%rax` and corresponds to `p[10]`.  the reason for +80 is that a single long takes eight bytes
* the next
```
    addq (%rdi), %rax
```
is a combined load and addition instruction that reads `p[0]` and adds it to `%rax`
* note that `*p` and `p[0]` end up using the same instruction



* pointers are always addresses regardless of their types
* let's do the same thing for pointers to `int` (32 bit integer)


In [ ]:
%%writefile array_index_int.c
int array_index_int(int * p) {
  return p[0] + p[10];
}

In [ ]:
gcc -O3 -S array_index_int.c
cat array_index_int.s


* the only differences that arise by the size of `int`, which is four bytes, are that
 * `movl` (a load instruction for Long word (4 bytes)) instruction is used instead of `movq`, and
 * the address accessed for `p[10]` is p + 40 (p + 4 * 10) instead of p + 80



* exactly the same for structures


In [ ]:
%%writefile struct_field.c
typedef struct {
  long x;
  long y;
  long z;
} point;
  
long struct_field(point * p) {
  return p->x + p->y + p->z;
}

In [ ]:
gcc -O3 -S struct_field.c
cat struct_field.s


* from the generated instructions, we can observe load instructions access
 * `(%rdi)` for `p->x`,
 * `8(%rdi)` for `p->y`, and 
 * `16(%rdi)` for `p->z`

* the reason why these addresses are eight bytes apart is that each of `x` and `y` is eight bytes large



* as you can imagine, the same for array of structures
* guess what kind of instructions are generated for the function below and see it


In [ ]:
%%writefile struct_array_field.c
typedef struct {
  long x;
  long y;
  long z;
} point;
  
long struct_array_field(point * p) {
  return p[10].x + p[10].y + p[10].z;
}

In [ ]:
gcc -O3 -S struct_array_field.c
cat struct_array_field.s


* you can observe that load instructions access
 * `240(%rdi)` for p->x,
 * `248(%rdi)` for p->y, and
 * `256(%rdi)` for p->z

* the result is expected, as the size of a structure is presumably 8 bytes x 3 = 24 bytes



* an example of dereferencing pointers multiple times
* guess what kind of instructions are generated for the function below and see it



In [ ]:
%%writefile ptr_ptr.c
typedef struct node {
  struct node * l;
  struct node * r;
} node;
  
node * left_right(node * n) {
  return n->l->r;
}

In [ ]:
gcc -O3 -S ptr_ptr.c
cat ptr_ptr.s


<a name="pointer_deref_assign"> </a>
## 7-2. pointer dereferencing + assignment

In [ ]:
%%writefile struct_array_field_assign.c
typedef struct {
  long x;
  long y;
  long z;
} point;
  
void struct_array_field(point * p) {
  p[10].x = 30;
  p[10].y = 40;
  p[10].z = 50;
}

In [ ]:
gcc -O3 -S struct_array_field_assign.c
cat struct_array_field_assign.s


* what corresponds to
```
    p[10].x = 30;
```
appears
```
    movq $30,240(%rdi)
```

* this is a _<font color="blue">store instruction</font>_ that stores 30 at the address `%rdi + 240`
* the name of the instruction (`movq`) is the same as load instruction, but as it is register $\rightarrow$ memory, it is a store instruction
* note that we compiled this example with `-O2` (lower optimization level), as using `-O3` uses an instruction (a SIMD instruction) in a way so specific to this code. give it a try if you are interested



<a name="problem_l2_norm_long"> </a>
# <font color="green"> Problem 2 :  square norm</font>
* write a function `l2_norm_long` in assembly that computes the square norm of three-element vector of longs

* in other words, write an assembly code corresponding to the following C function

```
long l2_norm_long(long * x) {
    return x[0] * x[0] + x[1] * x[1] + x[2] * x[2];
}
```

* _<font color="blue">hint:</font>_ if you need registers other than `%rdi`, `%rax`, use `%rdx`, `%rcx` etc. as necessary. see details at <a href="#registers">list of registers</a>


In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile l2_norm_long.s
	.file	"l2_norm_long.c"
	.text
	.p2align 4,,15
	.globl	l2_norm_long
	.type	l2_norm_long, @function
l2_norm_long:
.LFB0:
	.cfi_startproc
/* ------- 解答をここに書く. write your answer here ------- */

	.cfi_endproc
.LFE0:
	.size	l2_norm_long, .-l2_norm_long
	.ident	"GCC: (Ubuntu 7.4.0-1ubuntu1~18.04.1) 7.4.0"
	.section	.note.GNU-stack,"",@progbits


In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile check_l2_norm_long.c
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
long l2_norm_long(long *);

int main(int argc, char ** argv) {
  long x[3] = { atol(argv[1]), atol(argv[2]), atol(argv[3]) };
  long l2 = l2_norm_long(x);
  assert(l2 == x[0] * x[0] + x[1] * x[1] + x[2] * x[2]);
  printf("OK\n");
  return 0;
}

In [ ]:
BEGIN SOLUTION
END SOLUTION
gcc -o check_l2_norm_long -O3 check_l2_norm_long.c l2_norm_long.s

In [ ]:
BEGIN SOLUTION
END SOLUTION
./check_l2_norm_long 1 2 3
./check_l2_norm_long 3 4 5


<a name="passing_args"> </a>
# 8. Passing more arguments


* we have seen that the first parameter is put on `%rdi` but how about second and subsequent parameters

* let's see how it works by compiling a function taking six parameters


In [ ]:
%%writefile args.c
long add6(long a, long b, long c, long d, long e, long f) {
  return 1111 * a + 2222 * b + 3333 * c + 4444 * d + 5555 * e + 6666 * f;
}

In [ ]:
gcc -O3 -S args.c
cat args.s


* observing it with clues that it multiplies the first parameter by 1111 and the second by 2222, etc. we can deduce the following

|parameter number|location|
|--------|----|
| 1      |%rdi|
| 2      |%rsi|
| 3      |%rdx|
| 4      |%rcx|
| 5      |%r8|
| 6      |%r9|


* you can confirm this by <a href="https://github.com/hjl-tools/x86-psABI/wiki/x86-64-psABI-1.0.pdf">ABI specification</a> too, but also by writing a simple program like this



<a name="more_args"> </a>
## 8-1. more registers
* with only a small number of registers, there must be a limit on the number of parameters passed via registers
* as a matter of fact, seventh and subsequent parameters are passed differently


In [ ]:
%%writefile args_many.c
long add9(long a, long b, long c, long d, long e, long f,
          long g, long h, long i) {
  long x = 1111 * a;
  x += 2222 * b;
  x += 3333 * c;
  x += 4444 * d;
  x += 5555 * e;
  x += 6666 * f;
  x += 7777 * g;
  x += 8888 * h;
  x += 9999 * i;
  return x;
}

In [ ]:
gcc -O3 -S args_many.c
cat args_many.s


* you can deduce the following
 * the 7th parameter is at 8(%rsp)
 * the 8th parameter is at 16(%rsp)
 * the 9th parameter is at 24(%rsp)
`%rsp` is called a _<font color="blue">stack pointer</font>_ and adjusted upon a function call; details are described below

* at this point, understand that seventh and further parameters are arranged as above from the perspective of a called function



<a name="float_args"> </a>
## 8-2.  parameters of floating point numbers
* despite being numbers, floating point numbers use different instructions for operations and different registers as operands
* accordingly, ABI therefore specifies different ways to pass and return floating point numbers


In [ ]:
%%writefile fadd.c
double fadd(double a, double b) {
  return a + b + b;
}

In [ ]:
gcc -O3 -S fadd.c
cat fadd.s


* instructions
```
    addsd %xmm1, %xmm0  # %xmm0 += %xmm1
    addsd %xmm1, %xmm0  # %xmm0 += %xmm1
```
* what can be observed (inferred)
 * the first parameter (a) is on `%xmm0`
 * the second parameter (b) on `%xmm1`
 * the return value on `%xmm0`



* confirm with another example (multiplication)


In [ ]:
%%writefile fmul.c
double fmul(double a, double b, double c, double d) {
  return a * b + c * d;
}

In [ ]:
gcc -O3 -S fmul.c
cat fmul.s


* instructions
```
    mulsd %xmm1, %xmm0  # xmm0 *= xmm1 (a = a * b)
    mulsd %xmm2, %xmm3  # xmm3 *= xmm2 (d = c * d)
    addsd %xmm3, %xmm0  # xmm0 += xmm3 (a = a + d)
```
* observations
 * the first parameter `xmm0`
 * the second parameter `xmm1`
 * the third parameter `xmm2`
 * the fourth parameter `xmm3`
 * the return value `xmm0`



<a name="many_args"> </a>
## 8-3. when there are many parameters

In [ ]:
%%writefile fadd10.c
double fadd10(double a, double b, double c, double d, double e, double f,
            double g, double h, double i, double j) {
  return a + b + c + d + e + f + g + h + i + j;
}

In [ ]:
gcc -O3 -S fadd10.c
cat fadd10.s


* observations
 * the 1st parameter `%xmm0`
 * the 2nd parameter `%xmm1`
 * ...
 * the 7th parameter `%xmm6`
 * the 8th parameter `%xmm7`
 * the 9th parameter `8(%rsp)`
 * the 10th parameter `16(%rsp)`




<a name="many_types_of_args"> </a>
## 8-4. when integers, pointers, and float point numbers are mixed

In [ ]:
%%writefile args_mixed_many.c
long x;
double y;
long gg(double * A, double a, double * B, double b, double * C, double c,
        double * D, double d, double * E, double e, double * F, double f,
        double * G, double g, double * H, double h, double * I, double i,
        double * J, double j) {
  return A[0] + a + B[0] + b + C[0] + c + D[0] + d + E[0] + e
    + F[0] + f + G[0] + g + H[0] + h + I[0] + i + J[0] + j;
}

In [ ]:
gcc -O3 -S args_mixed_many.c
cat args_mixed_many.s


* generated instructions
```
    addsd (%rdi), %xmm0   # A[0] + a
    movq 8(%rsp), %rax    # %rax = G
    addsd (%rsi), %xmm0   # + B[0]
    addsd %xmm1, %xmm0    # + b
    addsd (%rdx), %xmm0   # + C[0]
    addsd %xmm2, %xmm0    # + c
    addsd (%rcx), %xmm0   # + D[0]
    addsd %xmm3, %xmm0    # + d
    addsd (%r8), %xmm0    # + E[0]
    addsd %xmm4, %xmm0    # + e
    addsd (%r9), %xmm0    # + F[0]
    addsd %xmm5, %xmm0    # + f
    addsd (%rax), %xmm0   # + G[0]
    movq 16(%rsp), %rax   # %rax = H
    addsd %xmm6, %xmm0    # + g
    addsd (%rax), %xmm0   # + H[0]
    movq 24(%rsp), %rax   # %rax = I
    addsd %xmm7, %xmm0    # + h
    addsd (%rax), %xmm0   # + I[0]
    movq 40(%rsp), %rax   # %rax = J
    addsd 32(%rsp), %xmm0 # + i
    addsd (%rax), %xmm0   # + J[0]
    addsd	48(%rsp), %xmm0 # + j
```
* the observed relationship between parameters and locations

|param|location  |
|----|----------|
| A  | %rdi     |
| a  | %xmm0    |
| B  | %rsi     |
| b  | %xmm1    |
| C  | %rdx     |
| c  | %xmm2    |
| D  | %rcx     |
| d  | %xmm3    |
| E  | %r8      |
| e  | %xmm4    |
| F  | %r9      |
| f  | %xmm5    |
| G  | 8(%rsp)  |
| g  | %xmm6    |
| H  | 16(%rsp) |
| h  | %xmm7    |
| I  | 24(%rsp) |
| i  | 32(%rsp) |
| J  | 40(%rsp) |
| j  | 48(%rsp) |

* to summarize how parameters are passed,

* __points you learned:__
 1. the first six integers and pointers are put on `%rdi`, `%rsi`, `%rdx`, `%rcx`, `%r8`, `%r9`
 1. the first eight floating point numbers are put on `%xmm0`, `%xmm1`, ..., `%xmm7`
 1. spilled parameters are put them in `8(%rsp)`, `16(%rsp)`, ... regardless of the type



<a name="registers"> </a>
# 9. List of registers
* for those who wonder how many registers are there, [the Brown University course material](https://cs.brown.edu/courses/cs033/docs/guides/x64_cheatsheet.pdf) lists all integer registers
* in addition, `%xmm0` ... `%xmm15` are floating point registers
* to summarize here are the available registers

|-|-|-|-|-|-|-|-|
|-------|-------|---------|--------|--------|--------|--------|--------|
| %rax  | %rbx  | %rcx    | %rdx   | %rdi   | %rsi   | %rbp   |        |
| %r8   | %r9   | %r10    | %r11   | %r12   | %r13   | %r14   | %r15   |
| %xmm0 | %xmm1 | %xmm2   | %xmm3  | %xmm4  | %xmm5  | %xmm6  | %xmm7  |
| %xmm8 | %xmm9 | %xmm10  | %xmm11 | %xmm12 | %xmm13 | %xmm14 | %xmm15 |
| %rsp  |%eflags | %rip   |        |        |        |        |        |

* of which
 * `%rsp` is the _<font color="blue">stack pointer</font>_ used to point to an end of the stack (described below) and cannot be used for arbitrary purposes
 * `%rip` is the _<font color="blue">instruction pointer</font>_ that holds the address from which the next instruction will be read
 * `%eflags` is the _<font color="blue">flag</font>_ register that holds the result of the last compare instruction.  it is not explicitly used in programs but is set by a compare (cmp) instruction and implicitly referenced by conditional branch instructions

* minor remarks
 * a single `%xmm` register is in fact able to hold two double (64 bit) values or 4 float (32 bit) values and there are (SIMD) instructions that can operate on all of them, but such details are not important in this exercise
 * more recent processors have `%ymm` registers that can hold twice as many (4 double or 8 float) values and even more recent ones have `%zmm` registers that can hold another twice as many (8 double or 16 float) values




<a name="control_flows"> </a>
# 10. Control flows
* how control flows such as if statements or loops (while or for) are compiled?



<a name="if_stmt"> </a>
## 10-1. if statement

In [ ]:
%%writefile add_or_mul_long.c
long add_or_mul_long(long x, long y, long z) {
  if (x < y) {
    return y + z;
  } else {
    return y * z;
  }
}

In [ ]:
gcc -O3 -S add_or_mul_long.c
cat add_or_mul_long.s


* processors have a compare instruction and conditional branch instructions that change the address of the next instruction depending on the result of the last compare instruction
* this illustrates how the above if statement gets compiled
```
    compare x and y
    conditionally jump to L if x < y
    compute y + z
L:
    compute y * z
```
* the actual result
```
    cmpq %rsi, %rdi  # compare %rdi(x) and %rsi(y) (computes rdi - rsi)
    jl .L2           # jump if less (jump to .L2 if rdi - rsi < 0)
    movq %rsi, %rax  # rax = rsi (y)
    imulq %rdx, %rax # rax *= rdx (z)
    ret
    .p2align 4,,10
    .p2align 3
.L2:
    leaq (%rsi,%rdx), %rax # rax = rsi (y) + rdx (z)
    ret
```
* two lines before label .L2
```
    .p2align 4,,10
    .p2align 3
```
are not instructions and can be ignored for the purpose of understanding how they work, but 
* to summarize, it ensures the address .L2 is a multiple of 16, inserting no-op instructions as necessary [details](https://sourceware.org/binutils/docs/as/P2align.html#P2align)



<a name="cmp_insn"> </a>
## 10-2. on cmp instruction
```
    cmpq %rsi,%rdi
```
basically compares `%rsi` (y) and `%rdi` (x)
* some remarks
 * as always `q` stands for quadword
* you can consider that `cmpq A,B` does what `subq A,B` does, except that the result goes to `%eflags` register not B (i.e., `%eflags = B - A`)
* conditional branches (many instructions starting with `j`) may jump to a specified label depending on  `%eflags` (whether it's $< 0$, $\leq 0$, $=0$, etc.)

|instruction | meaning                   | an equivalent C statement  |
|--------|-------------------------------|----------------------------|
|jl .Lx  | jump if less than             | if (%eflags < 0) goto .Lx  |
|jle .Lx | jump if less than or equal    | if (%eflags <= 0) goto .Lx |
|jg .Lx  | jump if greater than          | if (%eflags > 0) goto .Lx  |
|jge .Lx | jump if greater than or equal | if (%eflags >= 0) goto .Lx |
|je .Lx  | jump if equal                 | if (%eflags == 0) goto .Lx |
|jne .Lx | jump if equal                 | if (%eflags != 0) goto .Lx |

* [list of all jump instructions](http://unixwiz.net/techtips/x86-jumps.html)
* it's actually a bit inaccurate to say "`%eflags` holds the result of the subtraction"; as a matter of actual implementation, it only holds the sign of the result, whether it is zero, etc. as a set of bits.  the distinction does not matter much, however, as you never explicitly extract bits in `%eflags`
* another remark: there is a choice as to whether a comparison is made as signed integers (int, long, etc. in C) or as unsigned integers (unsigned int, unsigned long, etc. in C).  above instructions are for signed comparisons
* just for your information, below are instructions for comparing unsigned values


|instruction | meaning                   | an equivalent C statement  |
|--------|------------------------|----------------------------|
|jb .Lx  | jump if below          | if (%eflags < 0) goto .Lx  |
|jbe .Lx | jump if below or equal | if (%eflags <= 0) goto .Lx |
|ja .Lx  | jump if above          | if (%eflags > 0) goto .Lx  |
|jae .Lx | jump if above or equal | if (%eflags >= 0) goto .Lx |



<a name="cmp_floats"> </a>
## 10-3. comparing floating point numbers
* if you think you need different instructions to compare floating point numbers, you are getting used to assembly languages
* do not think you have to remember each of them, let `gcc -S` teach you instruction name and google it


In [ ]:
%%writefile add_or_mul_double.c
double add_or_mul_double(double x, double y, double z) {
  if (x < y) {
    return y + z;
  } else {
    return y * z;
  }
}

In [ ]:
gcc -O3 -S add_or_mul_double.c
cat add_or_mul_double.s


* if you look at generated instructions,

```
add_or_mul_double:
.LFB0:
    .cfi_startproc
    comisd %xmm0, %xmm1   # %eflags = xmm1 (y) - xmm0 (x)
    ja .L7                # if (%eflags (= y - x) > 0) goto .L7
    mulsd %xmm2, %xmm1    # y *= z
    movapd %xmm1, %xmm0   # xmm0 = y
    ret
.L7:
    addsd %xmm2, %xmm1    #y += z
    movapd %xmm1, %xmm0   # xmm0 = y
    ret
```

* you can easily imagine `comisd` is the comparison instruction
* if you google `comisd instruction` this mysterious instruction stands for _<font color="blue">COMpare Scalar Double</font>_
* just like `cmpq`, `comisd A,B` computes B - A and puts the result into `%eflags`



<a name="loops"> </a>
## 10-4. loops (while and for)
* in assembly languages, loops are made of comparison and conditional branches just as if statements are
* the following is compiled with `-O1`, as `-O3` produces code that is difficult to understand
* to be sure, you don't have to become able to understand all the generated code to make a compiler, but only have to know a general method/template to translate each construct 


In [ ]:
%%writefile fact.c
long fact (long n) {
  long i = 1;
  long p = 1;
  while (i <= n) {
    p = p * i;
    i = i + 1;
  }
  return p;
}

In [ ]:
gcc -O3 -S fact.c
cat fact.s


* comments to generated instructions

```
fact:
.LFB0:
    .cfi_startproc
    testq %rdi, %rdi   # %eflags = %rdi (n) & %rdi (n)
    jle .L4            # if (%eflags (= n) <= 0) goto .L4
    movl $1, %eax      # rax (p) = 1
    movl $1, %edx      # rdx (i) = 1
.L3:
    imulq %rdx, %rax   # p = p * i
    addq $1, %rdx      # i = i + 1
    cmpq %rdx, %rdi    # %eflags = rdi (n) - rdx (i)
    jge .L3            # if (%eflags (= n - i) >= 0) goto .L3
    rep ret
.L4:
    movl $1, %eax
 ret
```



* in general
```
while (condition) 
    S;
```
is equivalent to
```
    goto LC;
LB:
    S;
LC:
    c = condition;
    if (c) goto LB;
```
and you can translate $S$ and _condition_

* the following code is more common (both are correct)

```
    c = condition;
    if (!c) goto LE;
LB:
    S;
    c = condition;
    if (c) goto LB;
LE:
```



* for-statement has a different syntax but is a special case of while-statement

```
for (init ; condition; increment)
    S;
```

is equivalent to

```
init;
while (condition) {
    S;
    increment;
}
```



In [ ]:
%%writefile ax_b.c
double ax_b(double x0, double a, double b, long n) {
  double x = x0;
  for (long i = 0; i < n; i++) {
    x = a * x + b;
  }
  return x;
}

In [ ]:
gcc -O3 -S ax_b.c
cat ax_b.s


<a name="problem_sum_array_long"> </a>
# <font color="green"> Problem 3 :  sum of long arrays</font>
* write a function `sum_array(a, n)` that computes the sum of an n-element array of longs `a` in assembly
* see the following sum_array_c for the equivalent C function (your jobs is to write sum_array in assembly language equivalent to it)


In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile sum_array.s
	.file	"sum_array.c"
	.text
	.globl	sum_array
	.type	sum_array, @function
sum_array:
.LFB0:
	.cfi_startproc
/* ------- 解答をここに書く. write your answer here ------- */
        
	.cfi_endproc
.LFE0:
	.size	sum_array, .-sum_array
	.ident	"GCC: (Ubuntu 7.4.0-1ubuntu1~18.04.1) 7.4.0"
	.section	.note.GNU-stack,"",@progbits

In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile check_sum_array.c
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
long sum_array(long *, long);

long sum_array_c(long * a, long n) {
  long s = 0;
  for (long i = 0; i < n; i++) {
    s += a[i];
  }
  return s;
}

int main(int argc, char ** argv) {
  long n = argc - 1;
  long * a = (long *)malloc(sizeof(long) * n);
  for (long i = 0; i < n; i++) {
    a[i] = atol(argv[i + 1]);
  }
  long sa = sum_array(a, n);
  assert(sa == sum_array_c(a, n));
  printf("OK\n");
  return 0;
}

In [ ]:
BEGIN SOLUTION
END SOLUTION
gcc -o check_sum_array -O3 check_sum_array.c sum_array.s

In [ ]:
BEGIN SOLUTION
END SOLUTION
./check_sum_array 1 2 3 4 5
./check_sum_array 1 2 3 4 5 -6
./check_sum_array 1 -2 3 -4 5 -6 7


<a name="problem_max_array_double"> </a>
# <font color="green"> Problem 4 :  maximum value in double array</font>
* write a function `max_array(a, n)` that computes the maximum value of an n-element array of doubles `a` in assembly
* you may assume all elements are positive and return 0 if there are no elements (i.e., n = 0)
* see `max_array_c` for an equivalent C function


In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile max_array.s
	.file	"max_array.c"
	.text
	.globl	max_array
	.type	max_array, @function
max_array:
.LFB0:
	.cfi_startproc
/* ------- 解答をここに書く. write your answer here ------- */

	.cfi_endproc
.LFE0:
	.size	max_array, .-max_array
	.ident	"GCC: (Ubuntu 7.4.0-1ubuntu1~18.04.1) 7.4.0"
	.section	.note.GNU-stack,"",@progbits

In [ ]:
BEGIN SOLUTION
END SOLUTION
%%writefile check_max_array.c
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
double max_array(double *, long);

double max_array_c(double * a, long n) {
  double m = 0.0;
  for (long i = 0; i < n; i++) {
    if (a[i] > m) m = a[i];
  }
  return m;
}

int main(int argc, char ** argv) {
  long n = argc - 1;
  double * a = (double *)malloc(sizeof(double) * n);
  for (long i = 0; i < n; i++) {
    a[i] = atof(argv[i + 1]);
  }
  double sa = max_array(a, n);
  assert(sa == max_array_c(a, n));
  printf("OK\n");
  return 0;
}

In [ ]:
BEGIN SOLUTION
END SOLUTION
gcc -o check_max_array -O3 check_max_array.c max_array.s

In [ ]:
BEGIN SOLUTION
END SOLUTION
./check_max_array 1.2 2.3 3.4
./check_max_array 1.2 3.4 2.3
./check_max_array 3.4 1.2 2.3
./check_max_array


<a name="functions_that_call_functions"> </a>
# 11. A general mechanism behind function calls (stack)
* functions we have seen so far never call another function
* in them, it has been enough to carry out the specified computation, given which registers hold inputs and which register should hold the return value
* there are few more things you have to understand when the function calls another function
* let's consider the following


In [ ]:
%%writefile call_gg_hh.c 
long gg(long);
long hh(long);
long ff() {
  long x = gg(1);
  long y = hh(2);
  return x + y;
}

In [ ]:
gcc -O3 -S call_gg_hh.c 
cat call_gg_hh.cs


* consider you are at `-->`, the instruction right after `gg(1)` returns

```
    call gg@PLT
--> ...
```

* at this point the return value of `gg(1)` should be on `%rax`
* what to do next is to call `hh(2)`, but just calling it would obviously destroy the return value of `gg(1)`, as `hh(2)` too will put the return value on `%rax`
* it is therefore necessary to save the return value of `gg(1)` somewhere, before calling `hh(2)`
* a similar situation arises whenever a value (local variables, arrays, intermediate results of an expression, etc.) needs to be preserved _<font color="blue">across function calls</font>_
* where should we save such values? it must be regions that are not destroyed across function calls
* there are two such regions
 1. a memory area, called the _<font color="blue">stack</font>_
 1. _<font color="blue">callee-save registers</font>_ registers so designated by ABI that they ought to be preserved across function calls



<a name="stack"> </a>
## 11-1. stack region
* a stack region is a part of address space that is managed by a convention and used for holding values that cannot be held on registers
* as a basis, a designated register called `%rsp` always points to an end of the stack region
* the addresses equal to or higher than the address in `%rsp` ($a \geq$ `%rsp`) are the region "being used" and the addresses below `%rsp` ($a <$ `%rsp`) are free

<img src="img/stack_1.svg" /> 



<a name="callee_save_regs"> </a>
## 11-2. callee-save registers
* callee-save registers are registers so specified in ABI that they should be preserved across function calls
* in x86_64 ABI, following registers are designated as callee-save registers

|callee-save registers|
|----------|
|%rbx      |
|%rbp      |
|%r12      |
|%r13      |
|%r14      |
|%r15      |

* let's say a function `f` calls another function `g`
* if `f` wants to preserve a value across the call to `g()`, it could put it on a callee-save register instead of the stack 
* in that case, on the other hand, `f` needs to preserve the value originally in that register when `f` starts execution and restores it when `f` returns; the reason is obvious, as any caller of `f` also expects `f` to preserve values on callee-save registers as well (there are no free lunch)
* a function needs to use the stack region to save and restore callee-save registers
* a function usually saves and restores them upon entry to the function and return from it, respectively



* with that in mind, let's take another look at the compilation result of that function


In [ ]:
%%writefile call_gg_hh.c 
long gg(long);
long hh(long);
long ff() {
  long x = gg(1);
  long y = hh(2);
  return x + y;
}

In [ ]:
gcc -O3 -S call_gg_hh.c 
cat call_gg_hh.cs


* generated instructions
```
ff:
.LFB0:
    pushq %rbx      # save rbx (a callee-save register) on stack
    movl $1, %edi   # rdi = 1
    call gg@PLT     # gg(1)
    movq %rax, %rbx # save the return value on rbx
    movl $2, %edi   # rdi = 2
    call hh@PLT     # hh(2)
    addq %rbx, %rax # rax += rbx
    popq %rbx       # restore rbx
    ret
```

* here
```
    pushq %rbx
```
is an instruction that (1) decrements `%rsp` by eight (quadword) and stores the value of `%rbx` there.  it is equivalent to
```
    subq $8,%rsp
    movlq %rbx,(%rsp)
```
* recall that addresses below (lower than) `%rsp` are free, so `push X` essentially allocates a space for X and stores it there
* the operation is often expressed as _<font color="blue">"pushing `%rbx` onto the stack"</font>_
* on the contrary, 
```
    popq %rbx       # restore rbx
```
reads the value at the address pointed to by `%rsp` into `%rbx` and increments `%rsp` by eight.  it is equivalent to
```
    movlq (%rsp),%rbx
    addq $8,%rsp
```
* it is often expressed as _<font color="blue">"popping a value (from the stack) to `%rbx`"</font>_

* `ret` instruction is in fact an instruction that (1) pops an address from the stack and (2) jumps to the address.  semantically it is equivalent to this (remember that `%rip` is the register holding the address to fetch the next instruction from)
```
    popq %rip
```



* the above example used the stack merely to save/restore ``rbx`
* let's look at an example that uses the stack more extensively
* functions that use lots of values that must be preserved across function call tend to do so


In [ ]:
%%writefile many_vars.c
long gg(long);
long ff() {
  long a0 = gg(0);
  long a1 = gg(1);
  long a2 = gg(2);
  long a3 = gg(3);
  long a4 = gg(4);
  long a5 = gg(5);
  long a6 = gg(6);
  long a7 = gg(7);
  long a8 = gg(8);
  long a9 = gg(9);
  long x = gg(10);
  if (x < 0) return a0;
  if (x < 1) return a1;
  if (x < 2) return a2;
  if (x < 3) return a3;
  if (x < 4) return a4;
  if (x < 5) return a5;
  if (x < 6) return a6;
  if (x < 7) return a7;
  if (x < 8) return a8;
  return a9;
}

In [ ]:
gcc -O1 -S many_vars.c
cat many_vars.s


* observe
 * part of return values of `gg()` use callee-save registers
 * other values that do not fit callee-save registers use the stack



* you can observe the stack being used for large data structures such as arrays
* omitting details, the compilation flag `-fno-stack-protector` below simplifies the generated code


In [ ]:
%%writefile large_stack.c
long gg(long *);
long hh(long *);
long ff() {
  long a[100];
  long x = gg(a);
  long y = hh(a);
  return x + y;
}

In [ ]:
gcc -O1 -fno-stack-protector -S large_stack.c
cat large_stack.s


* observe
 * two callee-save registers, `%rbp` and `%rbx`, are used
 * they are accordingly `pushq`'ed prior to execution of the function
 * in addition, a memory block for `long a[100]` (800 bytes) is allocated on the stack
```
    subq $808, %rsp
```
* `%rbp` holds the address of `a`
* `%rbx` holds the return value of `gg` (`x`)



* x86_64 ABI specifies that during the execution of a function (right before it calls another function, more precisely), `%rsp` must be a multiple of sixteen
* in other words, at the beginning of a function, the remainder of `%rsp` divided by sixteen is eight (as the `call` instruction decrements rsp by eight)
* a function calling another function takes this into consideration when adjusting `%rsp`
* observe that after the above function executes `subq $808, %rsp`, `%rsp` is a multiple of sixteen (the remainder of `%rsp` is eight at the beginning; the function then performs two pushq's (rsp -= 16) and rsp -= 808;



<a name="summary_of_stack"> </a>
## 11-3. summary of how the stack works


say function `f` calls `g`

1.  right before the `call` instruction (`call g@PLT`)

<img src="img/stack_1.svg" /> 

2\. right after the `call` instruction = at the beginning of `g`

<img src="img/stack_2.svg" /> 

the `call` instruction decrements rsp and stores the address immediately following the call instruction at the end of the stack

3\. if `g` uses any callee-save register, it saves whatever are used on the stack with `pushq` (the stack right after those `pushq`s)

<img src="img/stack_3.svg" /> 

4\. if `g` needs more memory space on the stack it allocates the space by decrementing `%rsp` as necessary (the stack right after that)

<img src="img/stack_4.svg" /> 

5\. executes `g`

6\. when `g` returns, it deallocates the space by incrementing `%rsp` accordingly (the stack right after that)

<img src="img/stack_3.svg" /> 

7\. it restores callee-save registers by necessary `popq` instructions (the stack right after `popq`s)

<img src="img/stack_2.svg" /> 

8\. it executes `ret` instruction (the stack right after `ret`)

<img src="img/stack_1.svg" /> 
